In [1]:
import time
import matplotlib.pyplot as plt
import torch
import numpy as np
from laplacian_py import laplacian_solver

resx = 80
resy = 30
resz = 50

resx = 64
resy = 64
resz = 64

max_val = 100
cost_scale = .1

values = .99*max_val*torch.ones((resx, resy, resz),dtype=torch.float32, device='cuda')
cost = cost_scale*torch.ones((resx,resy, resz),dtype=torch.float32, device='cuda')
boundary_conditions = torch.zeros((resx,resy, resz),dtype=torch.float32, device='cuda')
boundary_types = torch.zeros((resx,resy, resz),dtype=torch.float32, device='cuda')
# goal
goal_ind1 = int(resx*15/30)
goal_ind2 = int(resy*15/30)
goal_ind3 = int(resz*15/30)
boundary_conditions[goal_ind1, goal_ind2, goal_ind3] = 0.0
boundary_types[goal_ind1, goal_ind2, goal_ind3] = 1.0



In [2]:
import sys
sys.path.append('../')

from scripts.point_cloud_utils import send_point_cloud
from pyngp.common import *
from tqdm import tqdm
import pyngp.pyngp as ngp
testbed = ngp.Testbed()
testbed.load_snapshot('../nerf/table.ingp')

box_diff = testbed.aabb.max - testbed.aabb.min
min_dim = -box_diff / 2
max_dim = box_diff / 2

x_segments = np.linspace(.2, .8, resx)
y_segments = np.linspace(.45-.02, .59-.02, resy)
z_segments = np.linspace(.35, .65, resz)
x, y, z = np.meshgrid(x_segments, y_segments, z_segments, indexing='ij')
points = np.stack((x.flatten(), y.flatten(), z.flatten()), axis=1)
points = points[0:256 * (points.shape[0] // 256), :]

dirs = np.zeros(points.shape)
dirs[:, 0] = 1
dirs[:, 1] = .5
dirs[:, 2] = .5

dt = np.zeros((points.shape[0], 1))
coords = np.hstack((points, dt, dirs))
nerf_values = testbed.sample_nerf(list(coords.flatten()))
nerf_values = np.reshape(nerf_values, (coords.shape[0], -1))

points = np.vstack((points[:, 0], points[:, 2], 1 - points[:, 1])).T
points = points * (max_dim - min_dim) + min_dim
points = 2 * points

send_point_cloud(np.hstack((points, nerf_values)), has_alpha=False, wait_time=5)

18:58:40 SUCCESS  Initialized CUDA 12.0. Active GPU is #0: NVIDIA GeForce RTX 4090 [89]
18:58:40 INFO     Loading network snapshot from: ../nerf/table.ingp
18:58:42 DEBUG    GPUMemory: allocating 1.41 KB.
18:58:42 DEBUG    GPUMemory: allocating 144 B.
18:58:42 DEBUG    GPUMemory: allocating 144 B.
18:58:42 DEBUG    GPUMemory: allocating 144 B.
18:58:42 DEBUG    GPUMemory: allocating 144 B.
18:58:42 DEBUG    GPUMemory: allocating 8 B.
18:58:42 DEBUG    GPUMemory: allocating 1.12 KB.
18:58:42 DEBUG    GPUMemory: allocating 4 MB.
18:58:42 DEBUG    GPUMemory: allocating 8 MB.
18:58:42 DEBUG    GPUMemory: allocating 2 MB.
18:58:42 DEBUG    GPUMemory: allocating 64 KB.
18:58:42 INFO     GridEncoding:  Nmin=16 b=2 F=4 T=2^22 L=8
18:58:42 DEBUG    GridEncoding at level 0: resolution=16 params_in_level=4096
18:58:42 DEBUG    GridEncoding at level 1: resolution=32 params_in_level=32768
18:58:42 DEBUG    GridEncoding at level 2: resolution=64 params_in_level=262144
18:58:42 DEBUG    GridEncoding 

In [3]:
nerf_values.shape[0]/60

4369.066666666667

In [4]:
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
import torch.optim as optim
from laplacian_py.network import LaplaceNet, compute_loss, calculate_gradient
    
network = LaplaceNet((resx, resy, resz), max_val, 1*cost_scale, 2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
network = network.to(device)

In [5]:
lr = 1e-2 # norm
# lr = 1e-4 # abs
momentum = 0.99
optimizer = optim.SGD(network.parameters(), lr=lr, momentum=momentum)

XYZ_train = [np.linspace(-1, 1, res, dtype=np.float32) for res in [resx, resy, resz]]
X_train, Y_train, Z_train = np.meshgrid(*XYZ_train)
out = np.stack([X_train, Y_train, Z_train], axis=3)
grid_train = torch.tensor(out, dtype=torch.float32, device='cuda', requires_grad=True)
grid_train = grid_train.unsqueeze(axis=0)

# target = torch.tensor(10+20*np.sqrt(X_train**2+Y_train**2+Z_train**2),dtype=torch.float32, device='cuda')
# target = torch.tensor(10+20*np.sqrt(X_train**2+Z_train**2),dtype=torch.float32, device='cuda')
target = torch.tensor(21+20*Y_train,dtype=torch.float32, device='cuda')

    
network.reset()
for it in range(0, 1000):
    optimizer.zero_grad()
    pred, C, _ = network(grid_train, boundary_types, boundary_conditions)
    loss_train = 1*compute_loss(pred=pred, target=target, C=C, cost_scale=cost_scale)
    if it % 10 == 0:
        print(loss_train)
    loss_train.backward()
    optimizer.step()
    
    
# network = torch.load('network_3d.pt')
    



/home/paul/CLionProjects/thesis_nerf/nerf_ws/install/laplacian_py/lib/python3.10/site-packages/laplacian_py/network.py:29: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  extra_cost = F.conv3d(objects_bounds.unsqueeze(dim=0), weight, bias=None, stride=1, padding='same',


tensor(6486.8105, device='cuda:0', grad_fn=<MulBackward0>)
tensor(760.6436, device='cuda:0', grad_fn=<MulBackward0>)
tensor(688.4364, device='cuda:0', grad_fn=<MulBackward0>)
tensor(728.6790, device='cuda:0', grad_fn=<MulBackward0>)
tensor(773.2325, device='cuda:0', grad_fn=<MulBackward0>)
tensor(780.5591, device='cuda:0', grad_fn=<MulBackward0>)
tensor(777.8230, device='cuda:0', grad_fn=<MulBackward0>)
tensor(774.6123, device='cuda:0', grad_fn=<MulBackward0>)
tensor(771.1303, device='cuda:0', grad_fn=<MulBackward0>)
tensor(767.4056, device='cuda:0', grad_fn=<MulBackward0>)
tensor(763.4558, device='cuda:0', grad_fn=<MulBackward0>)
tensor(759.3008, device='cuda:0', grad_fn=<MulBackward0>)
tensor(754.9581, device='cuda:0', grad_fn=<MulBackward0>)
tensor(750.4391, device='cuda:0', grad_fn=<MulBackward0>)
tensor(745.7643, device='cuda:0', grad_fn=<MulBackward0>)
tensor(740.9490, device='cuda:0', grad_fn=<MulBackward0>)
tensor(736.0565, device='cuda:0', grad_fn=<MulBackward0>)
tensor(731.20

In [6]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
import matplotlib
matplotlib.use('TkAgg')

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
                               
val = pred.cpu().detach().numpy()[1:-1,1:-1,1:-1]

skip = 2
color = val[0:-1:skip, 0:-1:skip, 0:-1:skip]

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(X_train[1:-1:skip,1:-1:skip,1:-1:skip], Y_train[1:-1:skip,1:-1:skip,1:-1:skip], Z_train[1:-1:skip,1:-1:skip,1:-1:skip],
          c=color)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()
plt.pause(0)

In [7]:
boundary_types_2 = boundary_types.clone()
boundary_conditions_2 = boundary_conditions.clone()


 
obj_inds = nerf_values[:,3] > 100/255
obj_inds = np.reshape(obj_inds, (resx,resy,resz))
boundary_conditions_2[obj_inds] = max_val*10
boundary_types_2[obj_inds] = 1.0


network.reset()
for i in range(1000):
    pred, C_pos, J2 = network(grid_train, boundary_types_2, boundary_conditions_2)
    V_in = pred.detach()
 



In [8]:
import rclpy
from rclpy.node import Node
from visualization_msgs.msg import MarkerArray, Marker
from geometry_msgs.msg import Point

if not rclpy.ok():
    rclpy.init()
node = Node('visualizer_node')
pub = node.create_publisher(MarkerArray, 'visualization_lines', 10)
    
msg = MarkerArray()

dt = 0.005
marker_id = 0
grid_pred = pred[1:-1, 1:-1, 1:-1].unsqueeze(axis=0).unsqueeze(axis=0)

x_min = points[0,0]
y_min = points[0,1]
z_min = points[0,2]
x_max = points[-1,0]
y_max = points[-1,1]
z_max = points[-1,2]

# x_segments = np.linspace(.2, .8, resx)
# y_segments = np.linspace(.45-.02, .59-.02, resy)
# z_segments = np.linspace(.35, .65, resz)


for z_val in [-.95, -.5, 0, .5 ,.95]:
    for y_val in [-.95, -.5, 0, .5 ,.95]:
        for x_val in [-.95, 0.0, .95]:
            cur = np.reshape(np.array([x_val, y_val, z_val]),(1, 1, 1, 1, 3))
            traj = np.zeros((1000, 3))
            for i in range(traj.shape[0]):
                query = torch.tensor(cur, dtype=torch.float32, device='cuda', requires_grad=True)
                query.unsqueeze(axis=0)
                J1, J2 = calculate_gradient(grid_pred, query)
                J2 = J2.cpu().detach().numpy()
                J2 = J2/(np.linalg.norm(J2) + .001)
                cur += -J2*dt
                traj[i, :] = cur.squeeze()
    
            marker = Marker()
            marker.header.frame_id = "unity";
            marker.id = marker_id;
            marker_id += 1
            marker.type = marker.LINE_STRIP;
            marker.action = marker.ADD;

            for ind in range(traj.shape[0]):
                position = Point()
                position.x = .5*(traj[ind, 2]+1.0)*(x_max-x_min) + x_min
                position.y = .5*(traj[ind, 1]+1.0)*(y_max-y_min) + y_min
                position.z = .5*(traj[ind, 0]+1.0)*(z_max-z_min) + z_min
                marker.points.append(position)

            marker.scale.x = .001;
            marker.scale.y = 0.0;
            marker.scale.z = 0.0;
            marker.color.a = 1.0; 
            marker.color.r = 0.0;
            marker.color.g = 0.0;
            marker.color.b = 1.0;

            msg.markers.append(marker)


pub.publish(msg)


In [9]:
for i in range(100):
    pub.publish(msg)



In [10]:
# dt = 0.01
# ax = plt.figure().add_subplot(projection='3d')

# grid_pred = pred[1:-1, 1:-1, 1:-1].unsqueeze(axis=0).unsqueeze(axis=0)

# for x_val in [-.95, -.5, 0, .5 ,.95]:
#     for y_val in [-.95, -.5, 0, .5 ,.95]:
#         for z_val in [-.95, 0.0, .95]:
#             cur = np.reshape(np.array([x_val, y_val, z_val]),(1, 1, 1, 1, 3))
#             traj = np.zeros((1000, 3))
#             for i in range(traj.shape[0]):
#                 query = torch.tensor(cur, dtype=torch.float32, device='cuda', requires_grad=True)
#                 query.unsqueeze(axis=0)
#                 J1, J2 = calculate_gradient(grid_pred, query)
#                 J2 = J2.cpu().detach().numpy()
#                 J2 = J2/(np.linalg.norm(J2) + .1)
#                 cur += -J2*dt
# #                 print(J2)
# #                 print(cur)
#                 traj[i, :] = cur.squeeze()

#             ax.plot(traj[:,0], traj[:,1], traj[:,2])
    
# obj_inds = (boundary_types_2 == 1).cpu().numpy()
# ax.scatter(X_train[obj_inds], Z_train[obj_inds],  Y_train[obj_inds])    
# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')
# plt.show()
# plt.pause(0)


In [11]:
# torch.save(network, 'network_3d.pt')